In [55]:
import pandas as pd
import os,glob
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append(os.path.expanduser("~") + "/collproc/")
from raw import *
from coll import *
import re

In [56]:
DIR = os.path.expanduser("~") + "/Dropbox/lml/genome_measurements/"

In [57]:
allthick = dirwalk(DIR,require='hic',omit=['_flag','thickness_charts.xls'])

In [58]:
df = pd.DataFrame()
for f in allthick:
    tmp = fproc(f)
    if tmp is not None:
        df = pd.concat([df,tmp])

In [59]:
df = df.reset_index(drop=True)

In [60]:
df.catalog.loc[df.fname.str.contains('ac_binder2_1')] = df.catalog.loc[df.fname.str.contains('ac_binder2_1')].apply(lambda x: f'{x.split("_")[1]}_{x.split("_")[0]}')

In [61]:
df['catalog_manbran'] = [item for item in df.catalog]

In [62]:
df['catalog'] = df.catalog.apply(str).apply(lambda x: x.split("_")[0])

### Legacy

In [75]:
legacy_file = "/Users/damoncrockett/Dropbox/lml/genome_measurements/genome_legacy/thickness_charts.xls"

In [76]:
lf = fproc(legacy_file)

In [77]:
lf = lf.reset_index(drop=True)

In [78]:
lf.catalog = lf.catalog.apply(str)
df.catalog = df.catalog.apply(str)

In [21]:
lf['catalog_manbran'] = [item for item in lf.catalog]

In [79]:
lf.catalog = lf.catalog.apply(lambda x: parse_sample_id(x)['catalog'])

# Legacy measurements never repeated

In [80]:
unique_trads = list(set(lf.catalog) - set(df.catalog))

In [81]:
# This is a check
# for unique_trad in unique_trads:
#     tmp = df[df.catalog.str.contains(unique_trad)]
#     if len(tmp) > 0:
#         print(unique_trad,"\n",sorted(list(tmp['catalog']),key=len),"\n\n\n\n")

In [82]:
len(unique_trads)

30

In [83]:
tf = pd.concat([df,lf])

# Overall standard deviation

In [84]:
# allmods = [item for sublist in df.thickness for item in sublist]
# alltrads = [item for sublist in lf.thickness for item in sublist]
# allthicknesses = allmods + alltrads

In [85]:
import numpy as np

In [86]:
# thickstd = np.std(allthicknesses)
# thickstd

# Modern dupes

#### preliminary dedupe

This is done for each measurement file individually, in the `raw` module, but we need to do it again here, to remove dupes that cross over files. There won't be tons of these, but I've already found one.

In [87]:
len(tf) - len(tf.drop_duplicates(subset=df.columns.difference(['thickness','fname','catalog_manbran','mera'])))

567

In [88]:
tf = tf.drop_duplicates(subset=df.columns.difference(['thickness','fname','catalog_manbran','mera']))
tf = tf.reset_index(drop=True)

In [89]:
tf.to_pickle('raw.pkl')